# FBRef Cleaning Notebook
**Author:** Jake Thomas
**Last Modified:** 12/10/2024 (created: 11/17/2024)

**Description:** Cleaning the FBref dataset and preparing it for merge.

In [40]:
# import libraries
import pandas as pd

In [41]:
fbref_df_raw = pd.read_csv("/Users/Jake/Documents/GitHub/Data-Wrangling-Project/data/raw/fbref_data_raw.csv", encoding="utf-8", index_col=False)

In [42]:
# Find the missing record for Marco Pellegrino
missing_record = fbref_df_raw[fbref_df_raw["Player"] == "Marco Pellegrino"]

# Impute missing values
fbref_df_raw.loc[fbref_df_raw["Player"] == "Marco Pellegrino", "Age"] = 22
fbref_df_raw.loc[fbref_df_raw["Player"] == "Marco Pellegrino", "Nationality"] = "ar ARG"

display(missing_record)

,Unnamed: 0,Player,Nationality,Position,Club Team,League,Age,Matches Played,Goals,Assists,Expected Goals
2071,2071,Marco Pellegrino,NaN,DF,Salernitana,it Serie A,NaN,10,0,0,0.0
2072,2072,Marco Pellegrino,NaN,DF,Milan,it Serie A,NaN,1,0,0,0.0


In [43]:
# Find duplicates
duplicates = fbref_df_raw[fbref_df_raw.duplicated(["Player", "Age"], keep=False)]

display(duplicates)


,Unnamed: 0,Player,Nationality,Position,Club Team,League,Age,Matches Played,Goals,Assists,Expected Goals
32,32,Lucien Agoume,fr FRA,MF,Inter,it Serie A,21.0,1,0,0,0.0
33,33,Lucien Agoume,fr FRA,MF,Sevilla,es La Liga,21.0,12,0,0,0.9
50,50,Sergio Akieme,es ESP,DF,Almería,es La Liga,25.0,19,2,0,1.2
51,51,Sergio Akieme,es ESP,DF,Reims,fr Ligue 1,25.0,13,2,0,1.4
54,54,Paul Akouokou,ci CIV,MF,Betis,es La Liga,25.0,2,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2828,2828,Alessio Zerbin,it ITA,"FW,DF",Napoli,it Serie A,24.0,7,0,0,0.2
2841,2841,Nadir Zortea,it ITA,DF,Frosinone,it Serie A,24.0,14,1,5,0.4
2842,2842,Nadir Zortea,it ITA,DF,Atalanta,it Serie A,24.0,5,1,0,0.1
2849,2849,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,33.0,20,5,1,4.5


In [44]:
# Add duplicate players statistics together
duplicates_copy = duplicates.copy()
duplicates_copy.reset_index(drop=True, inplace=True)


# Group by 'Player' and sum the specified columns
stats_to_sum = ["Goals", "Assists", "Matches Played", "Expected Goals"]
result = duplicates_copy.groupby("Player", as_index=False)[stats_to_sum].sum()

other_columns = duplicates_copy.drop(columns=stats_to_sum).drop_duplicates(subset=["Player"])
result = pd.merge(result, other_columns, on="Player", how="left")


display(result)


,Player,Goals,Assists,Matches Played,Expected Goals,Unnamed: 0,Nationality,Position,Club Team,League,Age
0,Abde Ezzalzouli,1,0,28,3.2,832,ma MAR,"FW,MF",Barcelona,es La Liga,21.0
1,Adam Masina,0,0,20,0.3,1647,ma MAR,DF,Udinese,it Serie A,29.0
2,Alan Virginius,0,2,16,1.3,2714,fr FRA,FW,Lille,fr Ligue 1,20.0
3,Alejo Véliz,1,0,14,0.4,2683,ar ARG,FW,Tottenham,eng Premier League,19.0
4,Alessandro Zanoli,0,2,21,0.2,2814,it ITA,DF,Salernitana,it Serie A,22.0
...,...,...,...,...,...,...,...,...,...,...,...
136,Tino Kadewere,1,0,25,2.6,1275,zw ZIM,"FW,MF",Lyon,fr Ligue 1,27.0
137,Tommaso Baldanzi,2,0,27,3.0,192,it ITA,"MF,FW",Empoli,it Serie A,20.0
138,Vitinha,12,6,55,9.3,2718,pt POR,"FW,MF",Genoa,it Serie A,23.0
139,Yann Karamoh,1,0,22,0.9,1295,fr FRA,"MF,FW",Torino,it Serie A,25.0


In [45]:
# Drop rows from fbref_df_raw that exist in duplicates based on a specific column (e.g., 'Player')
updated_df = fbref_df_raw[~fbref_df_raw['Player'].isin(duplicates['Player'])]

# Combine the remaining rows with the new rows
final_df = pd.concat([updated_df, result], ignore_index=True)

display(final_df)

,Unnamed: 0,Player,Nationality,Position,Club Team,League,Age,Matches Played,Goals,Assists,Expected Goals
0,0,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23.0,20,0,1,0.0
1,1,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22.0,30,2,2,2.0
2,2,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19.0,7,0,1,0.1
3,3,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17.0,1,0,0,0.0
4,4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35.0,31,4,0,3.4
...,...,...,...,...,...,...,...,...,...,...,...
2705,1275,Tino Kadewere,zw ZIM,"FW,MF",Lyon,fr Ligue 1,27.0,25,1,0,2.6
2706,192,Tommaso Baldanzi,it ITA,"MF,FW",Empoli,it Serie A,20.0,27,2,0,3.0
2707,2718,Vitinha,pt POR,"FW,MF",Genoa,it Serie A,23.0,55,12,6,9.3
2708,1295,Yann Karamoh,fr FRA,"MF,FW",Torino,it Serie A,25.0,22,1,0,0.9


In [46]:
# Impute missing values
final_df.loc[final_df["Player"] == "Max Moerstedt", "Age"] = 18

final_df.loc[final_df["Player"] == "Max Svensson", "Age"] = 23

final_df.loc[final_df["Player"] == "Atakan Karazor", "Nationality"] = "tr TUR"

final_df.loc[final_df["Player"] == "Mahmut Kücüksahin", "Nationality"] = "tr TUR"

display(final_df)

,Unnamed: 0,Player,Nationality,Position,Club Team,League,Age,Matches Played,Goals,Assists,Expected Goals
0,0,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23.0,20,0,1,0.0
1,1,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22.0,30,2,2,2.0
2,2,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19.0,7,0,1,0.1
3,3,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17.0,1,0,0,0.0
4,4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35.0,31,4,0,3.4
...,...,...,...,...,...,...,...,...,...,...,...
2705,1275,Tino Kadewere,zw ZIM,"FW,MF",Lyon,fr Ligue 1,27.0,25,1,0,2.6
2706,192,Tommaso Baldanzi,it ITA,"MF,FW",Empoli,it Serie A,20.0,27,2,0,3.0
2707,2718,Vitinha,pt POR,"FW,MF",Genoa,it Serie A,23.0,55,12,6,9.3
2708,1295,Yann Karamoh,fr FRA,"MF,FW",Torino,it Serie A,25.0,22,1,0,0.9


In [47]:
# Change age to int
final_df["Age"] = final_df["Age"].astype(int)
final_df.dtypes
display(final_df)

,Unnamed: 0,Player,Nationality,Position,Club Team,League,Age,Matches Played,Goals,Assists,Expected Goals
0,0,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,20,0,1,0.0
1,1,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,30,2,2,2.0
2,2,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,7,0,1,0.1
3,3,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,1,0,0,0.0
4,4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,31,4,0,3.4
...,...,...,...,...,...,...,...,...,...,...,...
2705,1275,Tino Kadewere,zw ZIM,"FW,MF",Lyon,fr Ligue 1,27,25,1,0,2.6
2706,192,Tommaso Baldanzi,it ITA,"MF,FW",Empoli,it Serie A,20,27,2,0,3.0
2707,2718,Vitinha,pt POR,"FW,MF",Genoa,it Serie A,23,55,12,6,9.3
2708,1295,Yann Karamoh,fr FRA,"MF,FW",Torino,it Serie A,25,22,1,0,0.9


In [48]:
# Save cleaned dataframe as new csv
final_df.to_csv("fbref_data_clean.csv", index=False, encoding="utf-8")